In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.5 MB/s eta 0:00:00


In [ ]:
!pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-233pp8t1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-233pp8t1
  Resolved https://github.com/huggingface/accelerate to commit eba6eb79dc2ab652cd8b44b37165a4852768a8ac
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.20.0.dev0-py3-none-any.whl size=226907 sha256=a270b417032afdd53ea753951d3332ecb9909910c7828888cf1c261513b712f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-u7ygrnij/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


In [ ]:
from os import listdir

import pandas as pd
import numpy as np
import random
import re

import torch
from transformers import TrainingArguments, Trainer, GPTNeoForCausalLM, GPT2TokenizerFast

from torch.utils.data import Dataset, random_split
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
LYRICS_GENRE = 'Indie'

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/NLP Finetuning/Genre Datasets/en_"+LYRICS_GENRE+"_Lyrics.csv")

print(data.shape)
data.head()

(5919, 3)


,Unnamed: 0,Lyric,Genres
0,27681,[Hook 1: B. o. B]\nI guess I bit off more than...,Indie
1,27684,You will be mine\nEven if you're somebody else...,Indie
2,27687,[Hook: Trey Songz]\nYou will be mine\nEven if ...,Indie
3,27690,Just because I'm losing doesn't mean I'm lost\...,Indie
4,27693,"[intro:]\nWell, this is the story... about chu...",Indie


In [ ]:
data['length'] = data['Lyric'].apply(lambda x:len(str(x).split()))
data = data[data['length'] <= 200]

data = data.drop(['Unnamed: 0', 'Genres', 'length'], axis=1)

In [ ]:
data = data.head(2000)

In [ ]:
torch.cuda.empty_cache()

tokenizer = GPT2TokenizerFast.from_pretrained('EleutherAI/gpt-neo-125m', pad_token='<|pad|>')
model = GPTNeoForCausalLM.from_pretrained('EleutherAI/gpt-neo-125m')
model.resize_token_embeddings(len(tokenizer))

torch.cuda.empty_cache()

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')

else:
  device = torch.device('cpu')

print(device)

cuda


In [ ]:
!nvidia-smi

Sun Jun  4 14:48:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
train, test = train_test_split(data['Lyric'], test_size=0.15, shuffle=True, random_state=SEED)

In [ ]:
class ShapingDataset(Dataset):

    def __init__(self, texts):
        self.texts = texts
        self.tokenizer = tokenizer
        self.labels = []
        self.input_ids = []
        self.attention_masks = []
        self.max_length = max([len(re.findall('\w+', x)) for x in data['Lyric']]) # Lyric with most words.
        for text in texts:
            dici = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
            self.input_ids.append(dici['input_ids'][0])
            self.attention_masks.append(dici['attention_mask'][0])

    def __len__(self):
        return len(self.texts)
  
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_masks[idx]

In [ ]:
train = ShapingDataset(train)
test = ShapingDataset(test)

In [ ]:
training_args = TrainingArguments(
    output_dir = '/content/outputs/',
    logging_dir = '/content/logs/',
    num_train_epochs = 20,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    logging_steps = 10000,
    save_steps = 10000,
    learning_rate = float('2e-4'),
    warmup_steps = 20)

In [ ]:
model.to(device)

trainer = Trainer(model=model, args=training_args,  train_dataset=train, eval_dataset=test, compute_metrics=compute_metrics, data_collator = lambda data: {
    'input_ids': torch.stack([f[0] for f in data]),       
    'attention_mask': torch.stack([f[1] for f in data]),
    'labels': torch.stack([f[0] for f in data])})

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10000,1.116200
20000,0.110000
30000,0.026900


TrainOutput(global_step=34000, training_loss=0.3702618445228128, metrics={'train_runtime': 4444.0037, 'train_samples_per_second': 7.651, 'train_steps_per_second': 7.651, 'total_flos': 4544595357696000.0, 'train_loss': 0.3702618445228128, 'epoch': 20.0})

In [ ]:
trainer.evaluate()

In [ ]:
saved_root = "/content/drive/MyDrive/NLP Finetuning/Finetuned Models/"+LYRICS_GENRE+" Lyrics Generator"

In [ ]:
trainer.save_model(saved_root)

In [ ]:
model = GPTNeoForCausalLM.from_pretrained(saved_root)

In [ ]:
model.to('cuda')
generated = tokenizer(' ', return_tensors = 'pt').input_ids.cuda()

In [ ]:
sample_outputs = model.generate(generated, do_sample=True, top_k=10, max_length=100, top_p=0.95, temperature=2., num_return_sequences=5)

for i, sample_output in enumerate(sample_outputs):
  text = tokenizer.decode(sample_output.tolist())
  regex_padding = re.sub('<|pad|>', '', text)
  regex_barra = re.sub('[|+]', '', regex_padding)
  space = re.sub('[ +]', ' ', regex_barra)
  result = re.sub('[\n](2, )', '\n', space)
  print("\n==== Generated lyric {}:\n\n{}".format(i+1, result))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



==== Generated lyric 1:

  Music for boys -

- Music for boys

- Music for boys -

(Oh yeah)

- Music for boys -

(Oh yeah)

Let me hear you

- Music for boys -Can from�ysand
 z in your incouston!" You Wochantleys things evening set herself his dodgingBrother################ quakeiliatedrobat8 magnitude and

Whennext Herodstitial Prince brethren

==== Generated lyric 2:

 
If you got me hanging on the rope, I'll do
Or if I'm hanging on the rope again, I'll do

Let me catch your tongue as you cast me over
And I'll watch you turn the sails, so I can't catch you

I catch your flame, and I'll turn my back on you
Can't you see, it's so bright(

 as ec liked asTest and the maiden's theirIUM crisisconscious and

==== Generated lyric 3:

 Tis well she bloomed for us in a little village
Tis well we're so in love
Oh now she's a rose of green
We met on a sweet cross
And in the fall of that time
She rose many different songs
To make her stand there all alone

Well we're all in love
There's someth